In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import seaborn as sns
cudnn.benchmark = True
import sklearn.metrics as metrics
from PIL import Image
from prettytable import PrettyTable
import cv2
from tqdm import tqdm

In [10]:
output_path = "Z://RR/Final/report_work/results/Trained_on_FairFace_Testing/FairFace_tests"

model_name = 'ResNet34_v21_5'
model_path = F"Z://RR/Final/report_work/saved_models/FairFace_All_Races_Testing/{model_name}.pt"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_path = 'Z://RR/Final/Datasets/FairFace_All_Races_Testing/test'


def show_image(file_path):
	# full_path = file_prefix + '/'+file_path
	img = cv2.imread(file_path)[:,:,::-1]
	plt.imshow(img)
	plt.show()
 
df_path ='Z://RR/FairFace/labels/fairface_label_full_allocated.csv'
FairFace_df = pd.read_csv(df_path)
# FairFace_df = FairFace_df[FairFace_df['Split'] == 'Test']


In [12]:
temp = FairFace_df[FairFace_df['Split'] == 'Train']
len(temp)

14655

In [11]:
FairFace_df.shape

(97698, 5)

In [7]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

data_transforms = {
	'test': transforms.Compose([ #compose several transforms together
		transforms.Resize(224),
		transforms.RandomHorizontalFlip(),
		transforms.ToTensor(),
		# transforms.Normalize(mean, std)
	]),
}

In [8]:
def load_checkpoint(model, load_path):
	if torch.cuda.is_available():
		
		checkpoint = torch.load(load_path)
	else:
		checkpoint = torch.load(load_path,map_location=torch.device('cpu'))
		
	model.load_state_dict(checkpoint['model_state_dict'])

	return model


def get_model_data(class_names):
	weights = models.ResNet34_Weights.DEFAULT
	model_ft = models.resnet34(weights = weights)
	num_ftrs = model_ft.fc.in_features
 
	model_ft.fc = nn.Linear(num_ftrs,len(class_names))
 
	model_ft= load_checkpoint(model_ft, model_path)
 
	model_ft = model_ft.to(device)
	model_ft.eval()
 
	return model_ft

def get_pred(model,file):
	img = Image.open(file)
	input = data_transforms['test'](img)
	input = input.unsqueeze(0)
	input = input.to(device)
	with torch.set_grad_enabled(False):
		output = model(input)
		_, preds = torch.max(output, 1)
	
	return preds[0].item() 
	

In [5]:
class_names = FairFace_df.age.value_counts().reset_index(name = "count").rename(columns={"index":"age"})['age']
model = get_model_data(class_names)

In [7]:
image_prefix = 'Z://RR/FairFace/images/archive/'

In [8]:

for index in tqdm(FairFace_df.index):
# for index in FairFace_df.index[:5]:
	
    
	path = os.path.join(image_prefix,FairFace_df.at[index,'file'])
	if os.path.isfile(path):
		class_pred = get_pred(model,path)
	
		FairFace_df.at[index,'prediction'] = class_pred

100%|██████████| 14655/14655 [31:18<00:00,  7.80it/s]


In [13]:
FairFace_df = FairFace_df[FairFace_df['prediction'].notna()]
# FairFace_df = FairFace_df[FairFace_df.prediction.map(type)==np.float64]

age_dist = FairFace_df.age.value_counts().reset_index(name = "count").rename(columns={"index":"age"})
age_dist.sort_values(by = ['age'],inplace=True)
age_dist = age_dist.reset_index(drop=True)
age_groups = age_dist['age']

In [14]:
def convert(pred):
    for i,group in enumerate(age_groups):
        if group == pred:
            return i

In [15]:
FairFace_df['pred_age_group'] = FairFace_df['prediction'].apply(lambda x: age_groups[x])
FairFace_df['true'] = FairFace_df['age'].apply(lambda x: convert(x))


In [16]:
FairFace_df.to_csv(output_path+'/FairFace_results.csv')